In [1]:
import pandas as pd
import numpy as np
import preprocess_helper
import os
from collections import Counter

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

import enchant
checker = enchant.Dict("en_US")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\11099\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Use the processed ASRS output from `load_asrs()` in `preprocess_helper.py`.

In [2]:
# Around 1000 samples 
ASRS = preprocess_helper.load_asrs(load_saved=True)

# Accident and incident counts of FAA and NTSB
acc_inc = pd.read_csv(r"E:\aviation_integrated\results\combined_vol_incident.csv")

# Dictionaries of acronym
# CASA = pd.read_csv('dictionaries/CASA.csv')
FAA = pd.read_csv('dictionaries/FAA.csv')
NASA = pd.read_csv('dictionaries/nasa_abbr.csv')
top_thirty = pd.read_csv('dictionaries/FAA Core Airports 2014.csv')

D:\Anaconda\envs\aviation_integrated\lib\site-packages\IPython\core\interactiveshell.py:3357: DtypeWarning: Columns (8,9,11,12,14,15,19,20,28,29,30,31,32,33,34,38,39,40,41,42,46,50,53,54,56,57,58,59,60,64,65,70,71,72,74,75,80,81,82,86,88,93,94,95,104) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


`narrative` is the combination of `narrative_report1` and `narrative_report2`.

In [3]:
ASRS[["narrative"]]

,narrative
0,we were in our clb from cmh to dfw with a clrn...
1,this was my first dep from bfi on 31l my turn ...
2,this was my first dep from bfi on 31l my turn ...
3,a vfr flt being conducted under far part 91 wa...
4,a vfr flt being conducted under far part 91 wa...
...,...
239242,on an a320 aircraft i installed a data loading...
239243,we departed out of atl behind an a320 and got ...
239244,ksfo quiet shoreline and rebas departures from...
239245,i took the day off from work yesterday to take...


In [4]:
ASRS.head()

,Unnamed: 0,ACN,Date,Local Time Of Day,Locale Reference,State Reference,Relative Position.Angle.Radial,Relative Position.Distance.Nautical Miles,Altitude.AGL.Single Value,Altitude.MSL.Single Value,...,Tower_ident_ct,FSS_ident_ct,Military Facility_ident_ct,CTAF_ident_ct,UNICOM_ident_ct,Ground_ident_ct,Ramp_ident_ct,AtcAdvisoryMultCount,year,month
0,1,79866.0,198801.0,1801-2400,CVG,OH,60.0,35.0,NaN,31000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1988.0,1.0
1,2,79912.0,198801.0,1201-1800,SEA; BFI,WA,NaN,6.0,NaN,900.0,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,2,1988.0,1.0
2,3,79912.0,198801.0,1201-1800,SEA; BFI,WA,NaN,6.0,NaN,900.0,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,2,1988.0,1.0
3,4,79965.0,198801.0,1201-1800,DEN,CO,NaN,20.0,NaN,7500.0,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,2,1988.0,1.0
4,5,79965.0,198801.0,1201-1800,DEN,CO,NaN,20.0,NaN,7500.0,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,2,1988.0,1.0


In [5]:
# Utility function used for counting the acronyms in ASRS data

# Check if a word is spelled correctly 
def spell_check(word):
    return checker.check(word)

# Count the total occurrences of all dictionary acronyms in the text 
def count_occurrence(word_count, dictionary):
    total = 0
    
    for word in dictionary:
        #if (word_count[word] != 0): print(word)
        total += word_count[word]
        
    return total

# Count the total unique occurrences of all dictionary acronyms in the text 
def count_unique(word_count, dictionary):
    total = 0
    
    for word in dictionary:
        if (word_count[word] != 0):
            total += 1
        
    return total

# Remove target symbols at both the start and the end of each string 
def strip_symbol(list_words, symbols):
    return list(map(lambda x: x.strip("',.:;"), list_words))

Note: It seems Jimmy had already removed all symbols in the `clean_string_and_tokenize` function.

**Update in 01/04/2022: Discard CASA dictionary temporarily and stop testing spelling check.** 

In [6]:
# Clean the field of acronyms

# CASA['acronym'] = CASA['acronym'].str.strip().str.lower()
# CASA['acronym'] = CASA['acronym'].str.replace(r'\(.+\)', '', regex=True)
# CASA.drop_duplicates(["acronym"], inplace=True, keep='first')
# CASA_cleaned = CASA[['acronym']]

FAA['acronym'] = FAA['acronym'].str.strip().str.lower()
FAA.drop_duplicates(["acronym"], inplace=True, keep='first')
FAA_cleaned = FAA[['acronym']]

NASA['acronym'] = NASA['acronym'].str.strip().str.lower()
NASA.drop_duplicates(["acronym"], inplace=True, keep='first')
NASA_cleaned = NASA[['acronym']]

top_thirty['tracon_key'] = top_thirty['tracon_key'].str.strip().str.upper()
top_thirty.drop_duplicates(["tracon_key"], inplace=True, keep='first')
top_thirty_cleaned = top_thirty[['tracon_key']]

# Remove stop words in the dictionaries
stop_words = set(stopwords.words('english'))

# bool_index = CASA_cleaned.isin(stop_words)['acronym'].values
# CASA_cleaned = CASA_cleaned[~bool_index]

bool_index = FAA_cleaned.isin(stop_words)['acronym'].values
FAA_cleaned = FAA_cleaned[~bool_index]

bool_index = NASA_cleaned.isin(stop_words)['acronym'].values
NASA_cleaned = NASA_cleaned[~bool_index]

# Filter out the words passing the spelling check (OBSOLETE)
# bool_index = np.array(list(map(spell_check, list(CASA_cleaned['acronym'].values))))
# CASA_cleaned = CASA_cleaned[~bool_index]

# bool_index = np.array(list(map(spell_check, list(FAA_cleaned['acronym'].values))))
# FAA_cleaned = FAA_cleaned[~bool_index]

# bool_index = np.array(list(map(spell_check, list(NASA_cleaned['acronym'].values))))
# NASA_cleaned = NASA_cleaned[~bool_index]

**Update in 01/07/2022: Use the top 30 airports listed in `FAA Core Airports 2014.csv`**

In [7]:
# Limit to top thirty airports
top_thirty_bool = ASRS['tracon_code'].isin(top_thirty['tracon_key'])
ASRS_30 = ASRS[top_thirty_bool]

# Count the occurrence of every distinct word in the text
words = ASRS_30['narrative'].str.lower().str.split()
words = words.apply(strip_symbol, symbols='{[(,:.')
words_count = words.apply(Counter)
acronyms = ASRS_30.loc[:, ['year', 'month', 'tracon_code', 'narrative','AtcAdvisoryMultCount']]
acronyms['word_counter'] = words_count

# Count the number of words
acronyms['word_count'] = words.apply(len)
acronyms['unique_word_count'] = words_count.apply(len)

# Sum the occurences of acronyms in different dictionaries respectively
# acronyms['CASA_total'] = acronyms['word_counter'].apply(count_occurrence, dictionary=CASA_cleaned['acronym'].values)
# acronyms['CASA_unique'] = acronyms['word_counter'].apply(count_unique, dictionary=CASA_cleaned['acronym'].values)

acronyms['FAA_total'] = acronyms['word_counter'].apply(count_occurrence, dictionary=FAA_cleaned['acronym'].values)
acronyms['FAA_unique'] = acronyms['word_counter'].apply(count_unique, dictionary=FAA_cleaned['acronym'].values)

acronyms['NASA_total'] = acronyms['word_counter'].apply(count_occurrence, dictionary=NASA_cleaned['acronym'].values)
acronyms['NASA_unique'] = acronyms['word_counter'].apply(count_unique, dictionary=NASA_cleaned['acronym'].values)

In [8]:
acronyms.head()
acronyms.to_csv("quality_check/full_count.csv")

In [9]:
acronyms.sort_values(['year', 'month', 'tracon_code'], inplace=True)
acronyms.head()

,year,month,tracon_code,narrative,AtcAdvisoryMultCount,word_counter,word_count,unique_word_count,FAA_total,FAA_unique,NASA_total,NASA_unique
146,1988.0,1.0,ATL,apching the atl area a solid line of thunderst...,1,"{'apching': 1, 'the': 13, 'atl': 4, 'area': 2,...",200,107,7,5,26,18
260,1988.0,1.0,ATL,my coplt and i mlg abc had just been clrd for ...,1,"{'my': 8, 'coplt': 1, 'and': 29, 'i': 21, 'mlg...",674,221,20,4,83,23
264,1988.0,1.0,ATL,while dsnding into the atl area on the 024 deg...,1,"{'while': 1, 'dsnding': 1, 'into': 1, 'the': 1...",173,102,5,2,25,17
15,1988.0,1.0,BOS,atis and arpt rwy condition broadcast was givi...,1,"{'atis': 1, 'and': 3, 'arpt': 2, 'rwy': 2, 'co...",60,48,1,1,6,4
84,1988.0,1.0,BOS,boston s logan arpt is very conscientious in t...,1,"{'boston': 2, 's': 1, 'logan': 1, 'arpt': 1, '...",169,103,0,0,12,6


In [35]:
# collapse by year-month-tracon and compute the sum and average per group
num_obs = pd.DataFrame(acronyms.groupby(['year', 'month', 'tracon_code']).size(), columns = ["num_obs"])
num_obs.reset_index(inplace = True)
sum_count = acronyms.groupby(['year', 'month', 'tracon_code']).sum()
sum_count.reset_index(inplace = True)
avg_count = acronyms.groupby(['year', 'month', 'tracon_code']).mean()
avg_count.reset_index(inplace = True)

agg_count = num_obs.merge(sum_count, on=['year', 'month', 'tracon_code'])
agg_count.rename(columns = {"word_count": "word_count_sum", "unique_word_count": "unique_word_count_sum",
                            "FAA_total": "FAA_total_sum", "FAA_unique": "FAA_unique_sum",
                            "NASA_total": "NASA_total_sum", "NASA_unique": "NASA_unique_sum",
                            "AtcAdvisoryMultCount": "AtcAdvisoryMultCount_sum"}, inplace=True)
agg_count = agg_count.merge(avg_count, on=['year', 'month', 'tracon_code'])
agg_count.rename(columns = {"word_count": "word_count_avg", "unique_word_count": "unique_word_count_avg",
                            "FAA_total": "FAA_total_avg", "FAA_unique": "FAA_unique_avg",
                            "NASA_total": "NASA_total_avg", "NASA_unique": "NASA_unique_avg",
                           "AtcAdvisoryMultCount": "AtcAdvisoryMultCount_avg", 'tracon_code': 'tracon'}, inplace=True)
agg_count

,year,month,tracon,num_obs,AtcAdvisoryMultCount_sum,word_count_sum,unique_word_count_sum,FAA_total_sum,FAA_unique_sum,NASA_total_sum,NASA_unique_sum,AtcAdvisoryMultCount_avg,word_count_avg,unique_word_count_avg,FAA_total_avg,FAA_unique_avg,NASA_total_avg,NASA_unique_avg
0,1988.0,1.0,ATL,3,3,1047,430,32,11,134,58,1.000000,349.000000,143.333333,10.666667,3.666667,44.666667,19.333333
1,1988.0,1.0,BOS,5,5,1088,587,24,11,82,46,1.000000,217.600000,117.400000,4.800000,2.200000,16.400000,9.200000
2,1988.0,1.0,CLT,1,1,294,139,10,2,37,17,1.000000,294.000000,139.000000,10.000000,2.000000,37.000000,17.000000
3,1988.0,1.0,DCA,5,6,1307,716,54,23,158,91,1.200000,261.400000,143.200000,10.800000,4.600000,31.600000,18.200000
4,1988.0,1.0,DEN,7,8,1735,877,71,32,234,113,1.142857,247.857143,125.285714,10.142857,4.571429,33.428571,16.142857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8692,2019.0,8.0,EWR,1,1,49,44,1,1,1,1,1.000000,49.000000,44.000000,1.000000,1.000000,1.000000,1.000000
8693,2019.0,8.0,LAX,1,1,465,234,2,2,15,7,1.000000,465.000000,234.000000,2.000000,2.000000,15.000000,7.000000
8694,2019.0,8.0,SFO,1,1,104,75,4,4,6,6,1.000000,104.000000,75.000000,4.000000,4.000000,6.000000,6.000000
8695,2019.0,9.0,ATL,1,1,180,126,5,4,5,4,1.000000,180.000000,126.000000,5.000000,4.000000,5.000000,4.000000


****
# Merge NTSB/FAA and ASRS (Temporarily obsolete)

In [32]:
# Limit NTSB/FAA to top 30
acc_inc_bool = acc_inc['airport_code'].isin(top_thirty['tracon_key'])
acc_inc_30 = acc_inc.loc[acc_inc_bool]
acc_inc_30 = acc_inc_30.drop("Unnamed: 0", axis = 1)
acc_inc_30 = acc_inc_30.rename({'airport_code': 'tracon'}, axis = 1)
acc_inc_30.to_csv('quality_check/acc_inc_30.csv')

In [12]:
final_merge = agg_count.merge(acc_inc_30, how = 'outer', on = ['year', 'month', 'tracon_code'])

In [17]:
final_merge.head(30)

,year,month,tracon_code,num_obs,AtcAdvisoryMultCount_sum,word_count_sum,unique_word_count_sum,FAA_total_sum,FAA_unique_sum,NASA_total_sum,...,FAA_total_avg,FAA_unique_avg,NASA_total_avg,NASA_unique_avg,ntsb_incidents,ntsb_accidents,faa_incidents,faa_ntsb_overlap,NTSB_FAA_incidents_total,NTSB_FAA_incidents_total_nodups
0,1988,1,MSP,1.0,2.0,321.0,186.0,2.0,1.0,31.0,...,2.0,1.0,31.0,19.0,0.0,0.0,3.0,0.0,3.0,3.0
1,1988,5,DFW,1.0,1.0,237.0,137.0,7.0,4.0,25.0,...,7.0,4.0,25.0,10.0,0.0,1.0,0.0,0.0,1.0,1.0
2,1988,6,DFW,1.0,1.0,57.0,45.0,3.0,3.0,10.0,...,3.0,3.0,10.0,9.0,0.0,0.0,1.0,0.0,1.0,1.0
3,1988,8,LAX,1.0,1.0,495.0,215.0,12.0,4.0,51.0,...,12.0,4.0,51.0,23.0,0.0,0.0,3.0,0.0,3.0,3.0
4,1988,10,MSP,1.0,1.0,128.0,72.0,4.0,3.0,11.0,...,4.0,3.0,11.0,9.0,0.0,0.0,1.0,0.0,1.0,1.0
5,1988,10,PHL,1.0,1.0,59.0,48.0,1.0,1.0,4.0,...,1.0,1.0,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN
6,1988,11,DCA,1.0,1.0,349.0,164.0,15.0,4.0,50.0,...,15.0,4.0,50.0,20.0,NaN,NaN,NaN,NaN,NaN,NaN
7,1989,1,DTW,1.0,1.0,71.0,48.0,3.0,2.0,7.0,...,3.0,2.0,7.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN
8,1989,5,SFO,1.0,1.0,366.0,188.0,24.0,8.0,49.0,...,24.0,8.0,49.0,22.0,NaN,NaN,NaN,NaN,NaN,NaN
9,1989,8,TPA,1.0,2.0,139.0,84.0,1.0,1.0,16.0,...,1.0,1.0,16.0,8.0,0.0,0.0,2.0,0.0,2.0,2.0


In [13]:
final_merge.to_csv("results/final_merge.csv")

****
## Create Master Key

In [26]:
date = pd.date_range(start = '1/1/1988', end = '12/31/2019', freq = 'M')
date_df = pd.DataFrame(data = {'year': date.year, 'month': date.month})
master_key = top_thirty.merge(date_df, how = 'cross')
master_key = master_key.drop('ATADS_Type', axis = 1)
master_key = master_key.rename({'tracon_key': 'tracon'}, axis = 1)
master_key

,tracon,year,month
0,ATL,1988,1
1,ATL,1988,2
2,ATL,1988,3
3,ATL,1988,4
4,ATL,1988,5
...,...,...,...
11515,TPA,2019,8
11516,TPA,2019,9
11517,TPA,2019,10
11518,TPA,2019,11


In [27]:
master_key.to_csv("quality_check/master_key.csv")

## Merge FAA/NTSB with master key

In [41]:
faa_ntsb_merge = master_key.merge(acc_inc_30, left_on=['tracon', 'year', 'month'], right_on=['tracon', 'year', 'month'], how='left')
faa_ntsb_merge.to_csv('quality_check/merge_acc_inc.csv')
faa_ntsb_merge

,tracon,year,month,ntsb_incidents,ntsb_accidents,faa_incidents,faa_ntsb_overlap,NTSB_FAA_incidents_total,NTSB_FAA_incidents_total_nodups
0,ATL,1988,1,0.0,0.0,1.0,0.0,1.0,1.0
1,ATL,1988,2,NaN,NaN,NaN,NaN,NaN,NaN
2,ATL,1988,3,1.0,0.0,4.0,1.0,5.0,4.0
3,ATL,1988,4,1.0,0.0,0.0,0.0,1.0,1.0
4,ATL,1988,5,0.0,2.0,0.0,0.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...
11515,TPA,2019,8,0.0,1.0,0.0,0.0,1.0,1.0
11516,TPA,2019,9,0.0,0.0,2.0,0.0,2.0,2.0
11517,TPA,2019,10,NaN,NaN,NaN,NaN,NaN,NaN
11518,TPA,2019,11,NaN,NaN,NaN,NaN,NaN,NaN


Note: Perhaps mark the missing data with an indicator column?

In [42]:
faa_ntsb_merge = final_merge.fillna(0)

## Merge ASRS with master key

In [47]:
agg_count.sort_values(['tracon', 'year', 'month']).to_csv("quality_check/agg_count.csv")

In [44]:
asrs_merge = master_key.merge(agg_count, left_on=['tracon', 'year', 'month'], right_on=['tracon', 'year', 'month'], how='left')
asrs_merge.to_csv('quality_check/merge_asrs.csv')
asrs_merge

,tracon,year,month,num_obs,AtcAdvisoryMultCount_sum,word_count_sum,unique_word_count_sum,FAA_total_sum,FAA_unique_sum,NASA_total_sum,NASA_unique_sum,AtcAdvisoryMultCount_avg,word_count_avg,unique_word_count_avg,FAA_total_avg,FAA_unique_avg,NASA_total_avg,NASA_unique_avg
0,ATL,1988,1,3.0,3.0,1047.0,430.0,32.0,11.0,134.0,58.0,1.0,349.000000,143.333333,10.666667,3.666667,44.666667,19.333333
1,ATL,1988,2,5.0,5.0,1610.0,811.0,74.0,31.0,200.0,105.0,1.0,322.000000,162.200000,14.800000,6.200000,40.000000,21.000000
2,ATL,1988,3,7.0,7.0,1250.0,770.0,30.0,13.0,141.0,79.0,1.0,178.571429,110.000000,4.285714,1.857143,20.142857,11.285714
3,ATL,1988,4,1.0,1.0,53.0,39.0,4.0,2.0,6.0,4.0,1.0,53.000000,39.000000,4.000000,2.000000,6.000000,4.000000
4,ATL,1988,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11515,TPA,2019,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11516,TPA,2019,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11517,TPA,2019,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11518,TPA,2019,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
asrs_merge  = asrs_merge.fillna(0)

## Create moving average of ASRS

In [99]:
asrs_splits = [asrs_merge[asrs_merge['tracon'] == tracon] for tracon in top_thirty['tracon_key']] # can be optimized
faa_ntsb_splits = [faa_ntsb_merge[faa_ntsb_merge['tracon'] == tracon] for tracon in top_thirty['tracon_key']] # can be optimized

### Toy example: ATL

In [124]:
ATL = asrs_splits[0]

In [125]:
window_len = 6
lag = 2
moving_avg = ATL.rolling(window_len, min_periods=window_len).mean()

In [126]:
ATL.head(10)

,tracon,year,month,num_obs,AtcAdvisoryMultCount_sum,word_count_sum,unique_word_count_sum,FAA_total_sum,FAA_unique_sum,NASA_total_sum,NASA_unique_sum,AtcAdvisoryMultCount_avg,word_count_avg,unique_word_count_avg,FAA_total_avg,FAA_unique_avg,NASA_total_avg,NASA_unique_avg
0,ATL,1988,1,3.0,3.0,1047.0,430.0,32.0,11.0,134.0,58.0,1.000000,349.000000,143.333333,10.666667,3.666667,44.666667,19.333333
1,ATL,1988,2,5.0,5.0,1610.0,811.0,74.0,31.0,200.0,105.0,1.000000,322.000000,162.200000,14.800000,6.200000,40.000000,21.000000
2,ATL,1988,3,7.0,7.0,1250.0,770.0,30.0,13.0,141.0,79.0,1.000000,178.571429,110.000000,4.285714,1.857143,20.142857,11.285714
3,ATL,1988,4,1.0,1.0,53.0,39.0,4.0,2.0,6.0,4.0,1.000000,53.000000,39.000000,4.000000,2.000000,6.000000,4.000000
4,ATL,1988,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,ATL,1988,6,5.0,5.0,1498.0,777.0,37.0,21.0,205.0,96.0,1.000000,299.600000,155.400000,7.400000,4.200000,41.000000,19.200000
6,ATL,1988,7,3.0,5.0,1072.0,478.0,28.0,11.0,127.0,58.0,1.666667,357.333333,159.333333,9.333333,3.666667,42.333333,19.333333
7,ATL,1988,8,8.0,11.0,2410.0,1212.0,50.0,30.0,286.0,158.0,1.375000,301.250000,151.500000,6.250000,3.750000,35.750000,19.750000
8,ATL,1988,9,5.0,6.0,1061.0,596.0,39.0,13.0,130.0,77.0,1.200000,212.200000,119.200000,7.800000,2.600000,26.000000,15.400000
9,ATL,1988,10,4.0,5.0,756.0,402.0,19.0,9.0,77.0,42.0,1.250000,189.000000,100.500000,4.750000,2.250000,19.250000,10.500000


In [127]:
moving_avg.head(10)

,year,month,num_obs,AtcAdvisoryMultCount_sum,word_count_sum,unique_word_count_sum,FAA_total_sum,FAA_unique_sum,NASA_total_sum,NASA_unique_sum,AtcAdvisoryMultCount_avg,word_count_avg,unique_word_count_avg,FAA_total_avg,FAA_unique_avg,NASA_total_avg,NASA_unique_avg
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1988.0,3.5,3.500000,3.500000,909.666667,471.166667,29.500000,13.000000,114.333333,57.000000,0.833333,200.361905,101.655556,6.858730,2.987302,25.301587,12.469841
6,1988.0,4.5,3.500000,3.833333,913.833333,479.166667,28.833333,13.000000,113.166667,57.000000,0.944444,201.750794,104.322222,6.636508,2.987302,24.912698,12.469841
7,1988.0,5.5,4.000000,4.833333,1047.166667,546.000000,24.833333,12.833333,127.500000,65.833333,1.006944,198.292460,102.538889,5.211508,2.578968,24.204365,12.261508
8,1988.0,6.5,3.666667,4.666667,1015.666667,517.000000,26.333333,12.833333,125.666667,65.500000,1.040278,203.897222,104.072222,5.797222,2.702778,25.180556,12.947222
9,1988.0,7.5,4.166667,5.333333,1132.833333,577.500000,28.833333,14.000000,137.500000,71.833333,1.081944,226.563889,114.322222,5.922222,2.744444,27.388889,14.030556


In [128]:
ATL.tail(10)

,tracon,year,month,num_obs,AtcAdvisoryMultCount_sum,word_count_sum,unique_word_count_sum,FAA_total_sum,FAA_unique_sum,NASA_total_sum,NASA_unique_sum,AtcAdvisoryMultCount_avg,word_count_avg,unique_word_count_avg,FAA_total_avg,FAA_unique_avg,NASA_total_avg,NASA_unique_avg
374,ATL,2019,3,1.0,1.0,308.0,169.0,8.0,5.0,11.0,8.0,1.0,308.0,169.0,8.0,5.0,11.0,8.0
375,ATL,2019,4,2.0,2.0,383.0,226.0,4.0,3.0,8.0,6.0,1.0,191.5,113.0,2.0,1.5,4.0,3.0
376,ATL,2019,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
377,ATL,2019,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
378,ATL,2019,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
379,ATL,2019,8,1.0,1.0,642.0,266.0,6.0,3.0,12.0,6.0,1.0,642.0,266.0,6.0,3.0,12.0,6.0
380,ATL,2019,9,1.0,1.0,180.0,126.0,5.0,4.0,5.0,4.0,1.0,180.0,126.0,5.0,4.0,5.0,4.0
381,ATL,2019,10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
382,ATL,2019,11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
383,ATL,2019,12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [129]:
moving_avg.tail(10)

,year,month,num_obs,AtcAdvisoryMultCount_sum,word_count_sum,unique_word_count_sum,FAA_total_sum,FAA_unique_sum,NASA_total_sum,NASA_unique_sum,AtcAdvisoryMultCount_avg,word_count_avg,unique_word_count_avg,FAA_total_avg,FAA_unique_avg,NASA_total_avg,NASA_unique_avg
374,2018.500000,6.5,1.000000,1.000000,468.166667,196.500000,8.833333,3.666667,11.166667,6.166667,0.666667,276.583333,121.666667,5.583333,2.416667,7.166667,4.083333
375,2018.666667,5.5,1.333333,1.333333,532.000000,234.166667,9.500000,4.166667,12.500000,7.166667,0.833333,308.500000,140.500000,5.916667,2.666667,7.833333,4.583333
376,2018.833333,4.5,1.166667,1.166667,498.333333,215.500000,8.500000,3.833333,11.166667,6.500000,0.666667,274.833333,121.833333,4.916667,2.333333,6.500000,3.916667
377,2019.000000,3.5,0.833333,0.833333,176.333333,98.666667,2.000000,1.333333,3.500000,2.500000,0.500000,113.833333,63.416667,1.666667,1.083333,2.666667,1.916667
378,2019.000000,4.5,0.833333,0.833333,176.333333,98.666667,2.000000,1.333333,3.500000,2.500000,0.500000,113.833333,63.416667,1.666667,1.083333,2.666667,1.916667
379,2019.000000,5.5,0.666667,0.666667,222.166667,110.166667,3.000000,1.833333,5.166667,3.333333,0.500000,190.250000,91.333333,2.666667,1.583333,4.500000,2.833333
380,2019.000000,6.5,0.666667,0.666667,200.833333,103.000000,2.500000,1.666667,4.166667,2.666667,0.500000,168.916667,84.166667,2.166667,1.416667,3.500000,2.166667
381,2019.000000,7.5,0.333333,0.333333,137.000000,65.333333,1.833333,1.166667,2.833333,1.666667,0.333333,137.000000,65.333333,1.833333,1.166667,2.833333,1.666667
382,2019.000000,8.5,0.333333,0.333333,137.000000,65.333333,1.833333,1.166667,2.833333,1.666667,0.333333,137.000000,65.333333,1.833333,1.166667,2.833333,1.666667
383,2019.000000,9.5,0.333333,0.333333,137.000000,65.333333,1.833333,1.166667,2.833333,1.666667,0.333333,137.000000,65.333333,1.833333,1.166667,2.833333,1.666667


In [130]:
ATL.columns

Index(['tracon', 'year', 'month', 'num_obs', 'AtcAdvisoryMultCount_sum',
       'word_count_sum', 'unique_word_count_sum', 'FAA_total_sum',
       'FAA_unique_sum', 'NASA_total_sum', 'NASA_unique_sum',
       'AtcAdvisoryMultCount_avg', 'word_count_avg', 'unique_word_count_avg',
       'FAA_total_avg', 'FAA_unique_avg', 'NASA_total_avg', 'NASA_unique_avg'],
      dtype='object')

In [131]:
moving_avg = ATL.rolling(window_len, min_periods=window_len).sum()
empty_row_df = pd.DataFrame(data = [pd.Series(index=ATL.columns) for _ in range(lag)])
empty_row_df = empty_row_df.drop('tracon', axis = 1)
moving_avg = pd.concat([empty_row_df, moving_avg])
moving_avg = moving_avg.drop(['year', 'month'], axis = 1)
moving_avg = moving_avg.reset_index().drop('index', axis = 1)
merge_df = pd.concat([faa_ntsb_splits[0], moving_avg], axis = 1)

D:\Anaconda\envs\aviation_integrated\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [132]:
merge_df.head(10)

,tracon,year,month,ntsb_incidents,ntsb_accidents,faa_incidents,faa_ntsb_overlap,NTSB_FAA_incidents_total,NTSB_FAA_incidents_total_nodups,num_obs,...,FAA_unique_sum,NASA_total_sum,NASA_unique_sum,AtcAdvisoryMultCount_avg,word_count_avg,unique_word_count_avg,FAA_total_avg,FAA_unique_avg,NASA_total_avg,NASA_unique_avg
0,ATL,1988.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ATL,1988.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ATL,1988.0,3.0,1.0,0.0,4.0,1.0,5.0,4.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ATL,1988.0,4.0,1.0,0.0,0.0,0.0,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ATL,1988.0,5.0,0.0,2.0,0.0,0.0,2.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,ATL,1988.0,6.0,0.0,0.0,2.0,0.0,2.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,ATL,1988.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,ATL,1988.0,8.0,3.0,0.0,3.0,3.0,6.0,3.0,21.0,...,78.0,686.0,342.0,5.000000,1202.171429,609.933333,41.152381,17.92381,151.809524,74.819048
8,ATL,1988.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,21.0,...,78.0,679.0,342.0,5.666667,1210.504762,625.933333,39.819048,17.92381,149.476190,74.819048
9,ATL,1988.0,10.0,0.0,0.0,2.0,0.0,2.0,2.0,24.0,...,77.0,765.0,395.0,6.041667,1189.754762,615.233333,31.269048,15.47381,145.226190,73.569048


In [137]:
merge_df

,tracon,year,month,ntsb_incidents,ntsb_accidents,faa_incidents,faa_ntsb_overlap,NTSB_FAA_incidents_total,NTSB_FAA_incidents_total_nodups,num_obs,...,FAA_unique_sum,NASA_total_sum,NASA_unique_sum,AtcAdvisoryMultCount_avg,word_count_avg,unique_word_count_avg,FAA_total_avg,FAA_unique_avg,NASA_total_avg,NASA_unique_avg
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11515,TPA,2019.0,8.0,0.0,1.0,0.0,0.0,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11516,TPA,2019.0,9.0,0.0,0.0,2.0,0.0,2.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11517,TPA,2019.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11518,TPA,2019.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Entire pipeline

In [158]:
window_len = 6
lag = 1

asrs_splits = [asrs_merge[asrs_merge['tracon'] == tracon] for tracon in top_thirty['tracon_key']] # can be optimized
faa_ntsb_splits = [faa_ntsb_merge[faa_ntsb_merge['tracon'] == tracon] for tracon in top_thirty['tracon_key']] # can be optimized
final = []

for i in range(len(asrs_splits)):
    asrs = asrs_splits[i].reset_index().drop('index', axis = 1)
    faa_ntsb = faa_ntsb_splits[i].reset_index().drop('index', axis = 1)
    
    # compute moving average of ASRS 
    mov_avg = asrs.rolling(window_len, min_periods=window_len).mean()
    
    # insert empty rows at the beginning for lagging
    empty_row_df = pd.DataFrame(data = [pd.Series(index=asrs.columns) for _ in range(lag)])
    empty_row_df = empty_row_df.drop('tracon', axis = 1)   
    
    # merge the moving averages of ASRS with FAA+NTSB 
    moving_avg = pd.concat([empty_row_df, mov_avg])
    moving_avg = moving_avg.drop(['year', 'month'], axis = 1)
    moving_avg = moving_avg.reset_index().drop('index', axis = 1)
    merge_df = pd.concat([faa_ntsb, moving_avg], axis = 1)
    final.append(merge_df)

final_df = pd.concat(final).reset_index().drop('index', axis = 1)
final_df.to_csv('quality_check/final.csv')

D:\Anaconda\envs\aviation_integrated\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  app.launch_new_instance()


In [159]:
final_df.head(10)

,tracon,year,month,ntsb_incidents,ntsb_accidents,faa_incidents,faa_ntsb_overlap,NTSB_FAA_incidents_total,NTSB_FAA_incidents_total_nodups,num_obs,...,FAA_unique_sum,NASA_total_sum,NASA_unique_sum,AtcAdvisoryMultCount_avg,word_count_avg,unique_word_count_avg,FAA_total_avg,FAA_unique_avg,NASA_total_avg,NASA_unique_avg
0,ATL,1988.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ATL,1988.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ATL,1988.0,3.0,1.0,0.0,4.0,1.0,5.0,4.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ATL,1988.0,4.0,1.0,0.0,0.0,0.0,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ATL,1988.0,5.0,0.0,2.0,0.0,0.0,2.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,ATL,1988.0,6.0,0.0,0.0,2.0,0.0,2.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,ATL,1988.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,3.500000,...,13.000000,114.333333,57.000000,0.833333,200.361905,101.655556,6.858730,2.987302,25.301587,12.469841
7,ATL,1988.0,8.0,3.0,0.0,3.0,3.0,6.0,3.0,3.500000,...,13.000000,113.166667,57.000000,0.944444,201.750794,104.322222,6.636508,2.987302,24.912698,12.469841
8,ATL,1988.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,4.000000,...,12.833333,127.500000,65.833333,1.006944,198.292460,102.538889,5.211508,2.578968,24.204365,12.261508
9,ATL,1988.0,10.0,0.0,0.0,2.0,0.0,2.0,2.0,3.666667,...,12.833333,125.666667,65.500000,1.040278,203.897222,104.072222,5.797222,2.702778,25.180556,12.947222


# Quality check

In [20]:
try:
    os.mkdir("quality_check")
except OSError as error:
    print(error)

np.random.seed(8)   
final_merge.sample(10).to_csv("quality_check/final_merge_10.csv")

[WinError 183] Cannot create a file when that file already exists: 'quality_check'
